In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [ ]:
from problems import *

In [ ]:
import os
from functools import partial
from natsort import natsorted
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
from scripts.create_problem import load_problem_set
import problems as pbs
import solvers as sv
from dpt.data import *
from dpt.reward import Reward, ZeroReward
from train_dpt import *
from notebooks.utils import *

In [ ]:
problem = 'Normal(0, 1)'
data_dir = '../../data/normal'
results_dir = '../../trajectories/normal'
suffix = 'val'

problems = load_problem_set(os.path.join(data_dir, problem, suffix))
dataset = OfflineDataset(problems, seq_len=5, results_dir=results_dir, suffix=suffix, action='point', target_action='next')
# dataset = OnlineDataset(problems)
sample = dataset[0]

for k, v in sample.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

In [ ]:
reward_model = ZeroReward()
rewards = reward_model.offline(sample['states'], sample['actions'], sample['next_states'])

In [ ]:
sample = dataset[0]
print_sample(sample, rewards, 'point')

In [ ]:
print(all(sample['target_action'][-1] == sample['target_state'][:-1]))

In [ ]:
collate_fn = partial(custom_collate_fn, problem_class=getattr(pbs, 'Problem'))
dataloader = DataLoader(dataset=dataset, batch_size=10, collate_fn=collate_fn)
batch = next(iter(dataloader))

for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

In [ ]:
rewards = reward_model.offline(
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"]
)
rewards.shape

In [ ]:
from train_dpt import DPTSolver, load_config

config = load_config('../configs/dpt_50.yaml')
# config['batch_size'] = 3
config['action'] = "point"
config['target_action'] = "next"
config['model_params']['action_dim'] = 50

# datamodule = ProblemDataModule(config)
# datamodule.setup()

# batch = next(iter(datamodule.test_dataloader()))

model = DPTSolver(config)
model.configure_model()

outputs = model.model(
    query_state=batch["query_state"],
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"],
    rewards=rewards,
)
outputs.shape

In [ ]:
# import matplotlib.pyplot as plt
# target_actions = [torch.argmax(sample['target_action']).item() for _ in range(100) for sample in dataset]
# bins = np.arange(0, 12)
# plt.hist(target_actions, bins-0.5)
# plt.xticks(bins[:-1])
# plt.show()